#Project 3: Assembly problem
JeongHo Choi, 20th June 2022 (updated)

-Download this FASTQ file containing synthetic sequencing reads from a mystery virus:
https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq<br>
-All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.<br>
-You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.<br>
-Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  
<br>
-Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

###Question 3)
How many As are there in the full, assembled genome?<br>
-Hint: the virus genome you are assembling is exactly 15,894 bases long

###Question 4) 
How many Ts are there in the full, assembled genome from the previous question?

In [1]:
from copy import deepcopy

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [2]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def kmers_dict(reads, k):
    kmers = {} # kmers dictionary; kmers as key, corresponding set of reads as value
    for read in reads:
        for i in range(len(read) - k + 1):  
            current_kmer = read[i:i+k]
            if current_kmer in kmers:
                kmers[current_kmer].add(read) # add a corresponding read to value if the kmer key exists
            else:
                kmers[current_kmer] = set([read]) # put a set of reads as value to avoid redundancy           
    return kmers

def pick_maximal_overlap_with_kmers(reads, k, kmers):
    reada, readb = None, None
    best_olen = 0
    for read in reads:
        current_suffix = read[len(read)-k:]
        reads_with_kmer = deepcopy(kmers[current_suffix]) # recall reads which include current_suffix as a kmer key
        reads_with_kmer.discard(read) # discard that read
        for read_with_kmer in reads_with_kmer:
            olen = overlap(read, read_with_kmer, k)
            if olen > best_olen:
                reada, readb = read, read_with_kmer 
                best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    kmers = kmers_dict(reads, k)
    read_a, read_b, olen = pick_maximal_overlap_with_kmers(reads, k, kmers)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        for key, value in kmers.items():
            for val in value.copy():
                if val == read_a or val == read_b:
                    value.discard(val) #if the read is repeated in kmers value, discard it
        read = read_a + read_b[olen:]
        for i in range(len(read) - k + 1): 
            current_kmer = read[i:i+k]
            if current_kmer in kmers:
                kmers[current_kmer].add(read)
            else:
                kmers[current_kmer] = set([read])
        read_a, read_b, olen = pick_maximal_overlap_with_kmers(reads, k, kmers)

    return ''.join(reads)

In [3]:
reads_filename = 'ads1_week4_reads.fq'
reads, _ = readFastq(reads_filename)
assembled_genome = greedy_scs(reads, 10)

# Q3
print("Number of As in the assembled genome:", assembled_genome.count('A'))

# Q4
print("Number of Ts in the assembled genome:", assembled_genome.count('T'))

Number of As in the assembled genome: 4633
Number of Ts in the assembled genome: 3723
